In [19]:
%load_ext autoreload
%autoreload 2
import os

from sentence_transformers import SentenceTransformer
from src.text_embedder.embedder import text_to_embedding_transformer

import pandas as pd
import torch
from loguru import logger
import pickle

from src.text_embedder.embedder import EmbeddingsModel

import tqdm
from pathlib import Path
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [20]:
notebook_path = Path().resolve().parent
print(notebook_path)

# Construct the filename relative to the new path
filename = notebook_path / "test_frames_gt_new.csv"

# Read the CSV file
df = pd.read_csv(filename)
print(df)

C:\Users\baatout\PycharmProjects\afm-vlm\experiments\second_experiment
    ID                                             Prompt Length  GT_Keyframe
0    1                                  learning outcomes  short            3
1    2   What are the learning outcomes from the lecture?   long            3
2    3       What is the relation of protien and muscles?  short           26
3    4                              Essential fatty acids  short           29
4    5                                  fish oil capsules  short           30
5    6  what is the functioning of fish oil capsules a...   long           30
6    7                           calcium as micronutrient  short           46
7    8       Explain where vitamins function as coenzymes   long           41
8    9  Give a list of food items where I can attain s...   long           45
9   10          what is calcium commonly supplemented for   long           46
10  11       How are nutrients extracted from plant cells   long       

In [21]:
# create dataframe to save results of the experiments
columns = ['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3']

df_test = pd.DataFrame(columns=columns)

df_ocr_only = df_test
df_ocr_lava = df_test
df_ocr_transcriptions = df_test

df_short_llm_summary = df_test
df_extensive_summary = df_test

df_clip_llm_summary = df_test
df_clip_extensive_summary = df_test

In [22]:
def add_to_df(prompt, gt_keyframe, result):
    # Create a new row with the provided data
    return {
        'Prompt': prompt,
        'GT_Keyframe': gt_keyframe,
        'Top_1': extract_keyframe_number(result[0]) if len(result) > 0 else None,
        'Top_2': extract_keyframe_number(result[1]) if len(result) > 1 else None,
        'Top_3': extract_keyframe_number(result[2]) if len(result) > 2 else None
    }


# ietrate over the dataframe and get the results
def get_results(df):
    for _, row in df.iterrows():
        logger.info(row['Prompt'])
        prompt = row['Prompt']
        gt_keyframe = row['GT_Keyframe']

        # Search for similar images
        output = embedding_model.search_similar_images_top_k(prompt, gt_keyframe, 3)
        res_row = add_to_df(prompt, gt_keyframe, output)
        rows.append(res_row)

    return rows


def extract_keyframe_number(path):
    """
    Extracts the scene number from the given file path.

    Parameters:
    path (str): The full path of the file.

    Returns:
    str: The extracted scene number.
    """
    # Get filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]

    # Extract '032' from filename
    scene_number = filename.split('-Scene-')[-1].split('-')[0]

    return scene_number

In [ ]:
#top_3 = []
#embedding_model.check_proximity_keyframes(90, top_3)

# LOAD MODEL 

In [23]:
embedding_model = EmbeddingsModel()

<Figure size 800x2000 with 0 Axes>

In [24]:
csv = notebook_path / "data_generation_pipeline_010724.csv"
df_data_extensive = pd.read_csv(csv)
df_data_extensive

,Unnamed: 0,79,4,67,16,75,8,20,43,32,...,70,13,62,1,54,29,37,46,58,25
0,img_path,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...,/Users/magic-rabbit/Documents/AFM/afm-vlm/data...
1,timestamps,"[3639.0, 3639.8]","[173.08, 174.08]","[3105.48, 3240.72]","[683.96, 734.36]","[3494.72, 3509.76]","[254.04, 374.08]","[846.68, 865.04]","[1860.88, 1932.44]","[1374.08, 1477.44]",...,"[3300.36, 3309.28]","[539.04, 607.32]","[2670.0, 2761.72]","[0.0, 158.72]","[2327.76, 2387.84]","[1224.24, 1339.64]","[1601.72, 1657.0]","[2000.2, 2062.64]","[2427.88, 2462.24]","[1086.24, 1118.2]"
2,transcription,NaN,NaN,"Well, whether ingested in food or supplemente...",Remember that carbohydrates are a macromolecu...,Here are some questions which process uses tr...,We studied water in our last chapter and we l...,Foods that haven't been stripped of their nut...,Water soluble vitamins are the ones that are ...,Here's some pictures to help you visualize th...,...,Facilitated diffusion is another way that sub...,Many people might choose to rehydrate themsel...,The mitochondria are an organelle that use gl...,"Welcome to Chapter 3, Nutrients, Membrings an...","As we learned in our last chapter, some membr...","Most mammals, including us humans, store fat ...","Unfortunately, this process creates something...",Calcium is a commonly supplemented mineral. Y...,"As you can see in the picture here, these pro...",Complete proteins contain all the essential a...
3,ocr_extracted_text,[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
4,llava_result,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The image you've provided appears to be a sli...,...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...,The slide appears to be from an academic lect...
5,clip_text,"Transport Across Membranes: Exocytosis, simple...",Supplements herbal FDA unapproved vitamins mem...,"Cell membrane transport, phospholipid bilayer ...",Carbohydrates macronutrient main energy source...,transport proteins facilitated diffusion plasm...,"Water disperses nutrients, dissolves waste pro...","Whole foods, unprocessed nutrients, macronutri...",NaN,"Saturated fats solid at room temp, contribute ...",...,"Passive transport, diffusion, facilitated diff...",Sports drinks unneeded calories additives unkn...,"Cell structure, organelles, mitochondria produ...","Vitamins essential micronutrients, coenzymes, ...",Cell membrane phospholipid bi

In [25]:
# get llm_long_summary 
llm_long_summary = df_data_extensive.iloc[6].to_dict().values()

# convert to list
llm_long_summary = list(llm_long_summary)

In [26]:
# get ocr_only
ocr_only = df_data_extensive.iloc[11].to_dict().values()

# convert to list
ocr_only = list(ocr_only)

In [27]:
# get ocr_lava ----- NOT EXISTING
ocr_lava = df_data_extensive.iloc[6].to_dict().values()

# convert to list
ocr_lava = list(ocr_lava)

In [28]:
# get ocr_transcriptions
ocr_transcriptions = df_data_extensive.iloc[14].to_dict().values()

# convert to list
ocr_transcriptions = list(ocr_transcriptions)

In [29]:
# get img_path
img_path =  df_data_extensive.iloc[0].to_dict().values()

# convert to list
img_path = list(img_path)
img_path

['img_path',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-079-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-004-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-067-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-016-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-075-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/raw/biology_chapter_3_3_treshhold_5/extracted_keyframes/biology_chapter_3_3_treshhold_5-Scene-008-01.jpg',
 '/Users/magic-rabbit/Documents/AFM/afm-vlm/data/ra

In [30]:
# Embedd with LLM long summary
logger.info(f"Embedd with LLM long summary")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(llm_long_summary)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_extensive_summary = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_extensive_summary.to_csv('df_standard_llm_long_summary.csv', index=False)


2024-07-03 19:57:30.171 | INFO     | __main__:<module>:2 - Embedd with LLM long summary
2024-07-03 19:57:38.983 | INFO     | __main__:<module>:18 - Embedding Model.text_embeddings: tensor([[-0.1943,  0.0935,  0.1131,  ..., -0.0911, -0.2710,  0.1646],
        [ 0.0253, -0.0263, -0.0014,  ...,  0.4360,  0.3577,  0.1273],
        [-0.3051, -0.3519, -0.0007,  ...,  0.1595,  0.0330,  0.1471],
        ...,
        [-0.0642, -0.2373,  0.4589,  ..., -0.1966,  0.1491, -0.1945],
        [-0.1577, -0.1107, -0.1429,  ...,  0.0462,  0.2383, -0.0868],
        [-0.2274, -0.1598,  0.0243,  ..., -0.0246,  0.0617, -0.0040]])
2024-07-03 19:57:38.983 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-03 19:57:39.030 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 19:57:39.030 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([86, 384])
2024-07-03 19:57:

<Figure size 800x2000 with 0 Axes>

In [31]:
# Embedd with OCR ONLY 
logger.info(f"Embedd with OCR ONLY ")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(ocr_only)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_ocr_only = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_only.to_csv('df_standard_ocr_only.csv', index=False)

2024-07-03 20:00:02.469 | INFO     | __main__:<module>:2 - Embedd with OCR ONLY 
2024-07-03 20:00:12.510 | INFO     | __main__:<module>:18 - Embedding Model.text_embeddings: tensor([[-0.0807, -0.2400,  0.0604,  ..., -0.1313,  0.0796,  0.0045],
        [-0.1382, -0.2408,  0.0148,  ..., -0.0026, -0.0469, -0.0778],
        [-0.1101, -0.2290,  0.0206,  ...,  0.0204,  0.0192, -0.1317],
        ...,
        [-0.1377, -0.2664,  0.0628,  ...,  0.0384,  0.0060, -0.0944],
        [-0.1165, -0.2758,  0.0403,  ...,  0.0028, -0.0008, -0.0745],
        [-0.0839, -0.2108,  0.1177,  ...,  0.0596, -0.0573, -0.1278]])
2024-07-03 20:00:12.510 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-03 20:00:12.557 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 20:00:12.557 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([86, 384])
2024-07-03 20:00:12.573 

<Figure size 800x2000 with 0 Axes>

In [32]:
# Embedd with OCR TRANSCRIPTIONS
logger.info(f"Embedd with TRANSCRIPTIONS")

result = []

embedding_model = EmbeddingsModel()

embedding_model.text_embeddings = None

embedding_model.img_paths = None

embedding_model.img_paths = img_path

embeddings = embedding_model.generate_dataset_embeddings_standard_tokenizer(ocr_transcriptions)

embedding_model.text_embeddings = embeddings

logger.info(f"Embedding Model.text_embeddings: {embedding_model.text_embeddings}")

rows = []

rows = get_results(df)

df_ocr_transcriptions = pd.DataFrame(rows, columns=['Prompt', 'GT_Keyframe', 'Top_1', 'Top_2', 'Top_3'])

# Save dataframe to CSV
df_ocr_transcriptions.to_csv('df_standard_df_ocr_transcriptions.csv', index=False)

2024-07-03 20:00:14.356 | INFO     | __main__:<module>:2 - Embedd with TRANSCRIPTIONS
2024-07-03 20:00:23.964 | INFO     | __main__:<module>:18 - Embedding Model.text_embeddings: tensor([[-0.1037, -0.2244, -0.1416,  ...,  0.0988,  0.2687, -0.2368],
        [-0.1382, -0.2408,  0.0148,  ..., -0.0026, -0.0469, -0.0778],
        [-0.1101, -0.2290,  0.0206,  ...,  0.0204,  0.0192, -0.1317],
        ...,
        [-0.1611, -0.2689,  0.0125,  ...,  0.0216, -0.0170, -0.0902],
        [-0.0916, -0.2179,  0.1103,  ...,  0.0467, -0.0785, -0.1586],
        [-0.0801, -0.1974,  0.0997,  ...,  0.0742, -0.0463, -0.1434]])
2024-07-03 20:00:23.964 | INFO     | __main__:get_results:15 - learning outcomes
2024-07-03 20:00:24.027 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:167 - Query text embedding shape: torch.Size([384])
2024-07-03 20:00:24.027 | INFO     | src.text_embedder.embedder:search_similar_images_top_k:168 - Text embeddings shape: torch.Size([86, 384])
2024-07-03 20:00:24

<Figure size 800x2000 with 0 Axes>